In [ ]:
import math
import os

In [ ]:
os.listdir('corpus1/spam')

## Email Corpus Preparation

In [ ]:
!git clone https://github.com/pachocamacho1990/datasets

Cloning into 'datasets'...
remote: Enumerating objects: 39, done.
remote: Total 39 (delta 0), reused 0 (delta 0), pack-reused 39
Unpacking objects: 100% (39/39), done.


In [ ]:
! unzip datasets/email/plaintext/corpus1.zip

In [ ]:
data = []
clases = []
#Spam Data
for file in os.listdir('corpus1/spam'):
  with open('corpus1/spam/'+file, encoding='latin-1') as f:
    data.append(f.read())
    clases.append('spam')
#Ham data
for file in os.listdir('corpus1/ham'):
  with open('corpus1/ham/'+file, encoding='latin-1') as f:
    data.append(f.read())
    clases.append('ham')
len(data)

5172

## Naive Bayes Model Building

### Spacy Tokenizer

* Docs: https://spacy.io/api/tokenizer
* ¿How its works? https://spacy.io/usage/linguistic-features#how-tokenizer-works

In [ ]:
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English

nlp = English()
tokenizer = Tokenizer(nlp.vocab)

In [ ]:
print([t.text for t in tokenizer(data[0])])

['Subject:', 'brand', 'new', 'teenager', 'peeing', '\n', 'you', 'don', "'", 't', 'know', 'me', 'from', 'adam', '.', ':', ')', 'iyakubonana', '\n', 'nothing', 'that', 'you', 'have', 'not', 'given', 'away', 'will', 'ever', 'be', 'really', 'yours', '.', '\n', 'i', 'don', "'", 't', 'deserve', 'this', 'award', ',', 'but', 'i', 'have', 'arthritis', ',', 'and', 'i', 'don', "'", 't', 'deserve', 'that', ',', 'either', '.', 'the', 'most', 'important', 'thing', 'a', 'father', 'can', 'do', 'for', 'his', 'children', 'is', 'to', 'love', 'their', 'mother', '.', '\n', 'tricks', 'and', 'treachery', 'are', 'the', 'practice', 'of', 'fools', ',', 'that', 'don', "'", 't', 'have', 'brains', 'enough', 'to', 'be', 'honest', '.', '\n', 'a', 'heart', 'that', 'loves', 'is', 'always', 'young', '.', '\n', 'if', 'you', 'drink', ',', 'don', "'", 't', 'drive', '.', 'don', "'", 't', 'even', 'putt', '.', '\n', 'husbands', 'never', 'become', 'good', 'they', 'merely', 'become', 'proficient', '.', '\n', 'only', 'from', 't

### Main Class for the Algorithm

Remember that the most probable class is given by (in logarithmic computation space):

$$\hat{c} = {\arg \max}_{(c)}\log{P(c)}
 +\sum_{i=1}^n
\log{ P(f_i \vert c)}
$$

Where, to avoid outliers, we will use Laplace smoothing as follows:

$$
P(f_i \vert c) = \frac{C(f_i, c)+1}{C(c) + \vert V \vert}
$$

with $\vert V \vert$ being the length of the vocabulary in our training set.


In [ ]:
import numpy as np

class NaiveBayesClassifier():
  nlp = English()
  tokenizer = Tokenizer(nlp.vocab)

  def tokenize(self, doc):
    return  [t.text.lower() for t in tokenizer(doc)]

  def word_counts(self, words):
    wordCount = {}
    for w in words:
      if w in wordCount.keys():
        wordCount[w] += 1
      else:
        wordCount[w] = 1
    return wordCount

  def fit(self, data, clases):
    n = len(data)
    self.unique_clases = set(clases)
    self.vocab = set()
    self.classCount = {} #C(c)
    self.log_classPriorProb = {} #P(c)
    self.wordConditionalCounts = {} #C(w|c)
    #Count clases
    for c in clases:
      if c in self.classCount.keys():
        self.classCount[c] += 1
      else:
        self.classCount[c] = 1
    # P(c)
    for c in self.classCount.keys():
      self.log_classPriorProb[c] = math.log(self.classCount[c]/n)
      self.wordConditionalCounts[c] = {}
    # C(w|c)
    for text, c in zip(data, clases):
      counts = self.word_counts(self.tokenize(text))
      for word, count in counts.items():
        if word not in self.vocab:
          self.vocab.add(word)
        if word not in self.wordConditionalCounts[c]:
          self.wordConditionalCounts[c][word] = 0.0
        self.wordConditionalCounts[c][word] += count

  def predict(self, data):
    results = []
    for text in data:
      words = set(self.tokenize(text))
      scoreProb = {}
      for word in words:
        if word not in self.vocab: continue #Ignor new words
        #Smooth P(w|c)
        for c in self.unique_clases:
          log_wordClassProb = math.log(
              (self.wordConditionalCounts[c].get(word, 0.0)+1)/(self.classCount[c]+len(self.vocab)))
          scoreProb[c] = scoreProb.get(c, self.log_classPriorProb[c]) + log_wordClassProb
      arg_maxprob = np.argmax(np.array(list(scoreProb.values())))
      results.append(list(scoreProb.keys())[arg_maxprob])
    return results


### Scikit Learn Utilities
* `train_test_split`: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

* `accuracy_score`: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html

* `precision_score`: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html

* `recall_score`: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
data_train, data_test, clases_train, clases_test = train_test_split(data, clases, test_size=0.10, random_state=42)

In [ ]:
classifier = NaiveBayesClassifier()
classifier.fit(data_train, clases_train)

In [ ]:
clases_predict = classifier.predict(data_test)

In [ ]:
accuracy_score(clases_test, clases_predict)

0.8552123552123552

In [ ]:
precision_score(clases_test, clases_predict, average=None, zero_division=1)

array([0.82876712, 1.        ])

In [ ]:
recall_score(clases_test, clases_predict, average=None, zero_division=1)

array([1.        , 0.51612903])